In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
# # Usar no Google Colab
# from google.colab import auth
# auth.authenticate_user()
# path_folder_bronze = '/content/'

In [ ]:
# Usar no VSCode
path_folder_silver = os.path.abspath('../data/temp/silver')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../gcp_key.json"

In [ ]:
def extract_year_month(date): # Função para extrair o ano e o mês de uma data
    return date.strftime('%y%m')

year_month = extract_year_month(pd.to_datetime('2017-06-01'))  # Exemplo de uso da função, como se fosse o dia 01 de junho de 2017
print(f'Year-Month: {year_month}')

In [ ]:
def read_csv_cleaned(path_folder_silver, file_name_silver): # Função para ler o arquivo CSV limpo
    df = pd.read_csv(os.path.join(path_folder_silver, file_name_silver))
    
    print(f'Arquivo "{file_name_silver}" lido.')
    return df

file_name_silver = f'mta_{year_month}_cleaned.csv'

df = read_csv_cleaned(path_folder_silver, file_name_silver)
display(df.head(15))
display(df.info())

In [ ]:
def remove_columns(df, columns_to_remove): # Função para remover colunas do DataFrame
    df = df.drop(columns=columns_to_remove)
    print(f'Colunas removidas: {columns_to_remove}')
    return df

df_ml = remove_columns(df, ['RecordedTimeRange', 'ScheduledTimeRange'])

display(df_ml.head(15))
display(df_ml.info())

In [ ]:
def remove_outliers_iqr(df, column_name): # Função para remover outliers usando o método IQR (Intervalo Interquartil)
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    print(f'Limite inferior: {lower_bound}, Limite superior: {upper_bound}')

    df = df[(df[column_name] > lower_bound) & (df[column_name] < upper_bound)]

    print(f'Outliers removidos da coluna "{column_name}" usando IQR.')
    return df


df_ml = remove_outliers_iqr(df_ml, 'DiffArrivalMins')

df_ml = df_ml.reset_index(drop=True)
display(df_ml.head(15))
display(df_ml.info())

In [ ]:
def transform_timedelta_to_seconds(df, column_name): # Função para transformar uma coluna de timedelta em segundos
    df[column_name] = pd.to_timedelta(df[column_name])
    df[column_name] = df[column_name].dt.total_seconds().astype(int)
    print(f'Coluna "{column_name}" transformada em segundos.')
    return df

df_ml = transform_timedelta_to_seconds(df_ml, 'RecordedAtTime')
df_ml = transform_timedelta_to_seconds(df_ml, 'ScheduledArrivalTime')

display(df_ml.head(15))

In [ ]:

def transform_date_to_days(df, column_name): # Função para transformar uma coluna de date em dias desde uma data de referência
    df[column_name] = pd.to_datetime(df[column_name])
    
    stdate = pd.Timestamp('2017-01-01')  # Data de referência para o cálculo dos dias
    
    df['RecordedAtDate'] = (df['RecordedAtDate'] - stdate).dt.total_seconds()
    df['RecordedAtDate'] = df['RecordedAtDate'] // (24 * 3600)  # Convertendo para dias
    df['RecordedAtDate'] = df['RecordedAtDate'].astype(int)
    
    print(f'Coluna "{column_name}" transformada para dias desde a data de referência.')
    return df

df_ml = transform_date_to_days(df_ml, 'RecordedAtDate')

display(df_ml.head(15))

In [ ]:
df_ml['PublishedLineName_hash'] = df_ml['PublishedLineName'].apply(lambda x: hash(x))
display(df_ml[['PublishedLineName', 'PublishedLineName_hash']].head(15))